# Data Supplementation (suppl)

Method
1.	Identify identity terms with the most disproportionate data distributions 
    1. Stem/lemmatize dataset
    2. For each lemma in the synthetic test set:
        1. Check distribution across labels in dataset, i.e. difference between frequency in toxic comments and overall
        2.	Also check length differences!
    3. What does this mean exactly? 
        1.	“Identity terms affected by the false positive bias are disproportionately used in toxic comments in our training data. For example, the word ‘gay’ appears in 3% of toxic comments but only 0.5% of comments overall.”
        2.	Frequency of identity terms in toxic comments and overall: 
2.	Add additional non-toxic examples that contain the identity terms that appear disproportionately across labels in the original dataset
    1.	Use wiki data – assumed to be non-toxic
    2.	Add enough so that the balance is in line with the prior distribution for the overall dataset
        1.	E.g. until % “gay” in toxic comment is close to 0.50% as in overall data.
3.	Maybe consider different lengths as CNNs could be sensitive to this
    1.	“toxic comments tend to be shorter” (Dixon et al. 2018)
4.	Supposed to reduce false positives. Could also do the opposite? But more difficult to find toxic comments unless we take them from places that are supposedly toxic (e.g. “roast me”)


## Imports

In [1]:
# set cwd
import os
os.chdir("g:\\My Drive\\ITC, 5th semester (Thesis)\\Code\\Github_code\\toxicity_detection")

# imports
import pandas as pd
# from random import choice, choices
# from collections import Counter
import matplotlib.pyplot as plt
# from string import punctuation
# # import spacy
from spacy import displacy
from tqdm import tqdm
from utils import load_dkhate
# from typing import Dict
import pickle
import dacy
# import utils
# import nltk
# import re
# import string
tqdm.pandas()

c:\Users\love2\anaconda3\envs\thesis2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Functions

In [2]:
def lemmatize_text(text:str) -> str:
    """Returns a lemmatized version of the text or itself if the string is empty."""
    if len(text) > 0:
        doc = nlp(text)
        lemmas = [token.lemma_ for token in doc]
        lemmatized_text = " ".join(lemmas)
        return lemmatized_text
    else:
        return text

def occurs_in(target, text):
    """Checks whether a word occurs in a text."""
    for word in text.split():
        if word == target:
            return 1
    return 0

## Load DaCy model

In [3]:
# load daCy model (medium works fine)
nlp = dacy.load("da_dacy_medium_trf-0.2.0") # takes around 4 minutes the first time

In [4]:
# test that it works as expected 
doc = nlp("Mit navn er Maja. Jeg bor på Bispebjerg, men er fra Næstved.") 
print("Token     \tLemma\t\tPOS-tag\t\tEntity type")
for tok in doc: 
    print(f"{str(tok).ljust(10)}:\t{str(tok.lemma_).ljust(10)}\t{tok.pos_}\t\t{tok.ent_type_}")
displacy.render(doc, style="ent")

Token     	Lemma		POS-tag		Entity type
Mit       :	Mit       	DET		
navn      :	navn      	NOUN		
er        :	være      	AUX		
Maja      :	Maja      	PROPN		PER
.         :	.         	PUNCT		
Jeg       :	jeg       	PRON		
bor       :	bo        	VERB		
på        :	på        	ADP		
Bispebjerg:	Bispebjerg	PROPN		LOC
,         :	,         	PUNCT		
men       :	men       	CCONJ		
er        :	være      	VERB		
fra       :	fra       	ADP		
Næstved   :	Næstved   	PROPN		LOC
.         :	.         	PUNCT		


## Load preprocessed training data

In [16]:
# load data splits 
_, _, y_train_orig, _ = load_dkhate(test_size=0.2)
with open(os.getcwd()+"/data/X_orig_preproc.pkl", "rb") as f:
    content = pickle.load(f)

X_train_orig = content["X_train"]
train_orig = pd.DataFrame([X_train_orig, y_train_orig]).T
train_orig.tail()

,tweet,label
id,,
2378,hørt,0
1879,reaktion svensker,0
42,hey champ smide link ser hearthstone henne,0
457,melder vold voldtægt viser sandt beviser diver...,1
3108,betaler omkring mb kb får nok tættere kb kb be...,0


In [6]:
# lemmatize the texts
train_orig["lemmas"] = train_orig["tweet"].progress_apply(lemmatize_text)

100%|██████████| 2631/2631 [06:10<00:00,  7.09it/s]


In [7]:
# split into toxic and all text
toxic_text = train_orig[train_orig["label"] == 1]["lemmas"]
all_text = train_orig["lemmas"]

NUM_TOXIC = len(toxic_text)
NUM_TOTAL = len(all_text)

toxic_text.head()

id
1174    scanne lortet pc markere tage underskrift ny d...
3301    kunne klarer fyr stort se venn vej samme spor ...
1390    fuck meget sol varme lille regn please dansk å...
799     hvorfor fucking stor helvede fejre kristn hell...
900     ingen udlænding ved grænse heller kriminell ku...
Name: lemmas, dtype: object

## Load identity terms

In [8]:
# load identity terms
identities = pd.read_excel(os.getcwd()+"/data/identity_terms.xlsx")
print(len(set(identities["identity_lemma"])), "unique identity lemmas")
identities.tail()

45 unique identity lemmas


,identity_term,identity_lemma
155,transpersonerne,transperson
156,transvestitterne,transvestit
157,transerne,trans
158,androgynerne,androgyn
159,hermafroditterne,hermafrodit


In [9]:
# lemmatize the identity terms
identities["lemmatized"] = identities["identity_term"].progress_apply(lemmatize_text)
print(len(set(identities["lemmatized"])), "unique lemmatized identity terms")
identities.tail()

100%|██████████| 160/160 [00:11<00:00, 14.45it/s]

133 unique lemmatized identity terms


,identity_term,identity_lemma,lemmatized
155,transpersonerne,transperson,transperson
156,transvestitterne,transvestit,transvestitterne
157,transerne,trans,transe
158,androgynerne,androgyn,androgynerne
159,hermafroditterne,hermafrodit,hermafroditterne


In [10]:
# create map from lemmatized word to the actual lemma
lemmatized_2_lemma = dict(zip(identities["lemmatized"], identities["identity_lemma"]))

## Perform Data Supplemenation

### Frequency of identity terms

In [11]:
# test function
print("This should return False. Result:", occurs_in("mor", "elsker din humor"))
print("This should return True.  Result:", occurs_in("mor", "hans mor er pænt sød"))

This should return False. Result: 0
This should return True.  Result: 1


In [12]:
# count how many texts these terms occur in
lemmatized_identities = list(set(identities["lemmatized"]))
occur_in_n_texts = {"lemmatized_identity": lemmatized_identities, "toxic_count": [], "total_count":[]}

for lemma in lemmatized_identities:
    occur_in_n_texts["toxic_count"].append(toxic_text.apply(lambda x: occurs_in(target=lemma, text=x)).sum())
    occur_in_n_texts["total_count"].append(all_text.apply(lambda x: occurs_in(target=lemma, text=x)).sum())

In [13]:
# create df with these occurrence numbers
occurrence_df = pd.DataFrame(occur_in_n_texts)

# map back to actual lemma and aggregate duplicates
occurrence_df["lemma"] = occurrence_df["lemmatized_identity"].map(lemmatized_2_lemma)
occurrence_df = occurrence_df.groupby("lemma").agg({"toxic_count": "sum", "total_count": "sum"}).reset_index()

# calculate percentages
occurrence_df["toxic_pct"] = (occurrence_df["toxic_count"]/NUM_TOXIC)*100 
occurrence_df["total_pct"] = (occurrence_df["total_count"]/NUM_TOTAL)*100 

# calculate differences
occurrence_df["difference"] = occurrence_df["toxic_pct"] - occurrence_df["total_pct"]
occurrence_df["abs_difference"] = abs(occurrence_df["toxic_pct"] - occurrence_df["total_pct"])

# sort by difference
sorted_occurrence_df = occurrence_df.sort_values("difference", ascending=False).reset_index(drop=True)

# display rows where toxic pct != total pct
sorted_occurrence_df[sorted_occurrence_df["difference"] != 0].round(2)

,lemma,toxic_count,total_count,toxic_pct,total_pct,difference,abs_difference
0,mand,16,73,4.60,2.77,1.82,1.82
1,kvinde,7,33,2.01,1.25,0.76,0.76
2,fyr,1,1,0.29,0.04,0.25,0.25
3,mandfolk,1,1,0.29,0.04,0.25,0.25
4,queer,1,1,0.29,0.04,0.25,0.25
5,kvindfolk,1,1,0.29,0.04,0.25,0.25
6,tøs,1,1,0.29,0.04,0.25,0.25
7,søn,1,2,0.29,0.08,0.21,0.21
8,fætter,1,3,0.29,0.11,0.17,0.17
9,kone,2,11,0.57,0.42,0.16,0.16


In [15]:
# save this df
sorted_occurrence_df.to_excel(os.getcwd()+"/mitigation/frequency_of_identity_lemmas.xlsx")

The ones with a difference > 0 are the ones that I need to look at. 

I can actually make a difference here by adding non-toxic data and getting the toxic_pct number closer to the total_pct number, thereby reducing the difference so it's as close to zero as possible. 

### Length differences

Percent of comments labeled as toxic at each length containing the given terms, e.g.:

| Term | 20-59 | 60-179 |
|:---:|:---:|:---:|
| ALL | 17% | 12% |
| gay | 88% | 77% |
| queer | 75% | 83% |
| ... | ... | ... |

Other lengths:
* 180-539
* 540-1619
* 1620-4859


Method:

* For each lemma:
  * Find the texts that it occur in
  * Separate these texts into 5 length buckets
  * For each length_bucket:
    * Find the percentage that are toxic